In [6]:
all_base_matrices = np.concatenate((pod.matrices, sym_nap_hex.matrices_hex), axis=0)[:, :-1, :-1]
all_labels = np.concatenate((pod.labels, sym_nap_hex.labels_hex))

In [7]:
trans_amount = len(pod.matrices) 
empty_arr = np.zeros((4,4))

In [8]:
full_list = np.zeros(len(sym_nap.all_matrix_translations) * len(pod.matrices) *16, dtype=int).reshape(-1, 4, 4)
full_list_labels = [""] * len(full_list)

In [ ]:
ID = np.eye(4)
for i in range(len(sym_nap.all_matrix_translations)):
    for j in range(len(pod.matrices)):
        combination = pod.matrices[j] + sym_nap.all_matrix_translations[i] - ID 
        place_in_array = trans_amount*i + j
        full_list[place_in_array] = combination
        name_m = pod.labels[j]
        name_t = sym_nap.all_labels_trans[i].lstrip("t")
        full_list_labels[place_in_array] = f'_matrix_{name_m}{name_t} = matrices_dict["{name_m}"] + _translation_{name_t}  #{place_in_array}'

print(len(full_list))

for mat in full_list_labels:
    print(mat, end='\n')

In [13]:
mydict_mats = {matrix.tostring(): label for matrix, label in zip(pod.matrices, pod.labels)}
mydict_trans = {tr.tostring(): label for tr, label in zip(sym_nap.all_translations, sym_nap.all_labels_trans)}

In [20]:
from collections import deque
from itertools import combinations, product

import podstawa as pod
import symetrie_napisane as sym_nap
# import symetrie_napisane_hex as sym_nap_hex
import symetrie_generowane as sym_gen
import numpy as np

def mult(M1, M2):
    M = M1 @ M2

    mask = M[:-1, -1] < 0
    M[:-1, -1][mask] += 12

    mask = M[:-1, -1] >= 12
    M[:-1, -1][mask] -= 12

    mask0 = M == 0
    M[mask0] -= 1
    M[mask0] += 1

    return M

def inverse(mat):
    new_mat = mat.copy()

    rot = new_mat[:-1, :-1]
    rot = rot.T

    trans = mat[:-1, -1] * -1
    mask_trans = trans < 0
    trans[mask_trans] += 12

    new_mat[:-1, :-1] = rot
    new_mat[:-1, -1] = trans

    return new_mat


def mat2name(mat, mapper):
    mat2 = inverse(mat)
    str_temp = mat.tostring()
    str_temp2 = mat2.tostring()
    val = mapper.get(str_temp)
    val2 = mapper.get(str_temp2)
    return val if val else val2

all_matrices_with_missing = sym_gen.all_missing_matrices + sym_nap.all_matrices
all_matrices_with_missing_labels = sym_gen.all_missing_matrices_labels + sym_nap.names

import pandas as pd

def translate_matrix_to_name():
    all_hashed_M = {M.tostring(): name for M, name in zip(all_matrices_with_missing, all_matrices_with_missing_labels)}
    l = np.zeros(len(all_hashed_M.keys()) * 16, dtype=int).reshape(-1, 4, 4)
    l_labels = [""] * len(l)
    for index, mat in enumerate(all_hashed_M.keys()):
        l[index] = np.frombuffer(mat, dtype=int).reshape(4,4)
    size = len(l)
    multi_table = np.empty((size, size), dtype=object)
    for ix in range(size):
        for iy in range(size):
            matrix = M(l[ix], l[iy])
            result = mat2name(matrix, all_hashed_M)
            multi_table[ix, iy] = result
    return multi_table, all_hashed_M

result, labels_from_function = translate_matrix_to_name()
data = pd.DataFrame(result)
data.columns = labels_from_function.values()
data.index = labels_from_function.values()
# data

class Queue:
    def __init__(self, *names):
        self.queue = deque([(el1, el2) for el1, el2 in product(names, repeat=2)])
        self.size = len(self.queue)

    def is_not_empty(self):
        return self.size != 0
        
    def enqueue(self, item):
        self.queue.append(item)
        self.size += 1 
    
    def add_combinations(self, addition):
        for name in self.uniques:
            self.enqueue((name, addition))
            self.enqueue((addition, name))
        self.enqueue((addition, addition)) 
            
    def dequeue(self):
        self.size -= 1
        return self.queue.popleft()
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.is_not_empty():
            return self.dequeue()
        else:
            raise StopIteration

class mycontainer:
    def __init__(self, table, *names):
        self.tablica = table
        self.queue = Queue(names)
        self.uniques = set(names)
        self.origin = names
#         self.calc()
        
    def calc(self):
        for name1, name2 in self.queue:
            wynik = self.tablica[name1][name2]
            if wynik not in self.uniques:
                self.expand(wynik)

    def expand(self, item):
        self.queue.add_combinations(item)
        self.uniques.add(item)
    
    def __repr__(self):
        return str(self.queue)
   
    def __contains__(self, item):
        return item in self.uniques

class mycontainer:
    def __init__(self, table, *names):
        self.tablica = table
        self.queue = deque([(el1, el2) for el1, el2 in product(names, repeat=2)])
        self.size = len(self.queue)
        self.uniques = set(names)
        self.origin = names
#         self.calc()
        
    def calc(self):
        for name1, name2 in self.queue:
            wynik = self.tablica[name1][name2]
            if wynik not in kolejka:
                kolejka.expand(wynik)

    def is_not_empty(self):
        return self.size != 0
    
    def expand(self, item):
        self.add_combinations(item)
        self.uniques.add(item)
    
    def add_combinations(self, addition):
        for name in self.uniques:
            self.enqueue((name, addition))
            self.enqueue((addition, name))
        self.enqueue((addition, addition)) 
    
    def __repr__(self):
        return str(self.queue)
    
    def enqueue(self, item):
        self.queue.append(item)
        self.size += 1 
        
    def dequeue(self):
        self.size -= 1
        return self.queue.popleft()
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.is_not_empty():
            return self.dequeue()
        else:
            raise StopIteration
    
    def __contains__(self, item):
        return item in self.uniques

from time import time

class Grupa:
    def __init__(self, konstruktory, all_syms):
        self.konstruktory = konstruktory
        self.all_syms = set(all_syms)
    
    def __repr__(self):
        return  ' '.join((str(self.konstruktory), str(self.all_syms)))
    
    def __eq__(self, other):
        return self.all_syms == other.all_syms
    
    def __hash__(self):
        return hash(tuple(sorted(self.all_syms)))
    
    @classmethod
    def from_queue(cls, queue):
        return cls(queue.origin, queue.uniques)
        

class Timer:
    def __init__(self):
        self.start = time()        
        self.mid = time()
        self.prev = 0
    
    def __call__(self, num):
        if num == self.prev:
            starter = round((time() - self.start) / 60, 1)
            mider = round(time() - self.mid, 1)
            self.mid = time()
            print(self.prev, starter, mider)
            self.prev += 1
            

zespol = [set() for _ in range(500)]

mytimer = Timer()
# kolejak osobno kurde 
for num1, num2 in combinations(range(len(data.columns)), 2):
    mytimer(num1)
    
    name1, name2 = [data.columns[num1], data.columns[num2]] 
    tabliczka = mycontainer(data, name1, name2)
    tabliczka.calc()
    num = len(tabliczka.uniques)
    zespol[num].add(Grupa.from_queue(tabliczka))
#     print(num, zespol)
    break
